In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable as V
import torch.nn.functional as F
from  sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.ensemble import IsolationForest
import pandas as pd
from torch.utils.data import DataLoader,Dataset,TensorDataset
import torch.optim as optim
import torch.nn.init as init
import numpy
from sklearn.model_selection import train_test_split

In [27]:
# load in datasets
RNAseq=pd.read_csv("/Users/vivit/CC/data_rnaSeq_G.csv")
train_clinical=pd.read_csv("/Users/vivit/CC/train_clinical_G.csv")
test_clinical=pd.read_csv("/Users/vivit/CC/test_clinical_G.csv")

In [28]:
train_idx=train_clinical[['Unnamed: 0.1']]
test_idx=test_clinical[['Unnamed: 0.1']]

In [29]:
RNAseq

,Unnamed: 0,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,AAA1,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
0,TCGA.CS.4938,6.5715,0.0000,4.5641,3.8564,7.1997,13.8518,6.2176,0.0000,0.0000,...,6.0266,9.0339,10.3581,0.0000,10.5476,10.6438,10.1957,9.7094,9.6052,0.0
1,TCGA.CS.4941,6.1944,0.0000,9.0375,7.1808,9.0290,14.1314,7.3288,1.0252,0.0000,...,6.7406,8.8721,10.2618,0.4277,10.7750,11.9259,10.7387,9.9894,4.3204,0.0
2,TCGA.CS.4942,6.2375,0.0000,8.5295,5.7117,7.4545,14.2341,5.5011,0.4287,0.0000,...,6.8608,8.8308,10.1078,0.4287,10.6322,11.1134,10.4766,9.4674,10.7374,0.0
3,TCGA.CS.4943,4.9535,0.0000,5.5235,3.9052,7.4920,13.5168,5.1915,1.2043,0.0000,...,6.4921,8.9427,10.3324,2.0911,10.7823,11.0584,10.9054,9.4311,8.0716,0.0
4,TCGA.CS.4944,4.6844,0.0000,6.7335,4.2491,7.3253,13.4115,6.8579,0.5892,0.0000,...,5.4385,8.3434,9.9855,0.5892,10.2598,10.7259,9.5133,8.8390,4.3694,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,TCGA.WY.A85A,6.3097,0.0000,7.4914,4.8743,8.3211,13.1848,5.7910,0.6115,0.0000,...,5.7772,8.4595,10.3039,0.0000,10.5437,10.5493,10.6046,9.3784,3.0686,0.0
504,TCGA.WY.A85B,6.4383,0.0000,5.4604,4.9876,5.3276,14.1384,5.5559,1.4442,0.0000,...,5.9364,8.9773,10.3973,0.8958,10.4965,10.6219,10.7201,9.7107,12.8674,0.0
505,TCGA.WY.A85C,6.9881,0.4944,5.5221,4.5847,5.1757,13.4168,6.5510,1.1545,0.0000,...,6.2746,9.0549,10.6047,1.9488,10.8586,10.9134,10.9709,9.8315,11.5579,0.0
506,TCGA.WY.A85D,8.2396,0.0000,4.7356,3.9758,8.0754,13.5098,6.0992,0.0000,0.0000,...,5.1703,8.2906,10.5089,0.0000,10.2572,10.7373,10.4823,9.5928,10.6111,0.0


In [30]:
RNAseq.shape

(508, 20119)

In [31]:
trainset = RNAseq.merge(train_idx,left_on='Unnamed: 0', right_on='Unnamed: 0.1',how="inner")
testset = RNAseq.merge(test_idx,left_on='Unnamed: 0', right_on='Unnamed: 0.1',how="inner")

In [32]:
trainset

,Unnamed: 0,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,AAA1,...,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR,Unnamed: 0.1
0,TCGA.CS.4941,6.1944,0.0000,9.0375,7.1808,9.0290,14.1314,7.3288,1.0252,0.0000,...,8.8721,10.2618,0.4277,10.7750,11.9259,10.7387,9.9894,4.3204,0.0,TCGA.CS.4941
1,TCGA.CS.4942,6.2375,0.0000,8.5295,5.7117,7.4545,14.2341,5.5011,0.4287,0.0000,...,8.8308,10.1078,0.4287,10.6322,11.1134,10.4766,9.4674,10.7374,0.0,TCGA.CS.4942
2,TCGA.CS.4943,4.9535,0.0000,5.5235,3.9052,7.4920,13.5168,5.1915,1.2043,0.0000,...,8.9427,10.3324,2.0911,10.7823,11.0584,10.9054,9.4311,8.0716,0.0,TCGA.CS.4943
3,TCGA.CS.4944,4.6844,0.0000,6.7335,4.2491,7.3253,13.4115,6.8579,0.5892,0.0000,...,8.3434,9.9855,0.5892,10.2598,10.7259,9.5133,8.8390,4.3694,0.0,TCGA.CS.4944
4,TCGA.CS.5393,5.5662,0.0000,8.3424,4.9114,7.3717,14.0085,6.2716,0.0000,0.0000,...,8.6205,10.3663,0.0000,10.6882,10.7732,10.9516,9.7250,9.7346,0.0,TCGA.CS.5393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,TCGA.WY.A858,7.7583,0.0000,2.6741,5.3648,5.5743,15.3476,7.9331,0.0000,0.0000,...,8.2489,10.0079,0.6763,9.8439,11.4981,10.3045,8.8761,9.2236,0.0,TCGA.WY.A858
351,TCGA.WY.A85A,6.3097,0.0000,7.4914,4.8743,8.3211,13.1848,5.7910,0.6115,0.0000,...,8.4595,10.3039,0.0000,10.5437,10.5493,10.6046,9.3784,3.0686,0.0,TCGA.WY.A85A
352,TCGA.WY.A85B,6.4383,0.0000,5.4604,4.9876,5.3276,14.1384,5.5559,1.4442,0.0000,...,8.9773,10.3973,0.8958,10.4965,10.6219,10.7201,9.7107,12.8674,0.0,TCGA.WY.A85B
353,TCGA.WY.A85C,6.9881,0.4944,5.5221,4.5847,5.1757,13.4168,6.5510,1.1545,0.0000,...,9.0549,10.6047,1.9488,10.8586,10.9134,10.9709,9.8315,11.5579,0.0,TCGA.WY.A85C


In [33]:
train_idx_73=trainset[["Unnamed: 0"]]

In [34]:
train_clinical73=train_clinical.merge(train_idx_73,left_on="Unnamed: 0.1",right_on="Unnamed: 0",how="inner")

In [35]:
train_clinical73

,Unnamed: 0_x,Unnamed: 0.1,years_to_birth,histological_type,gender,radiation_therapy,race,ethnicity,overall_survival,status,overallsurvival,Unnamed: 0_y
0,474,TCGA.TQ.A7RJ,25.0,oligoastrocytoma,female,yes,white,hispanicorlatino,1229.0,0.0,"1229,0",TCGA.TQ.A7RJ
1,227,TCGA.HT.7468,30.0,oligodendroglioma,male,no,white,nothispanicorlatino,203.0,0.0,"203,0",TCGA.HT.7468
2,52,TCGA.DB.A75P,25.0,astrocytoma,female,no,NaN,NaN,492.0,0.0,"492,0",TCGA.DB.A75P
3,320,TCGA.HT.A619,51.0,oligodendroglioma,female,no,asian,nothispanicorlatino,651.0,0.0,"651,0",TCGA.HT.A619
4,221,TCGA.FG.A711,33.0,oligodendroglioma,female,no,white,nothispanicorlatino,1481.0,1.0,"1481,1",TCGA.FG.A711
...,...,...,...,...,...,...,...,...,...,...,...,...
350,98,TCGA.DU.7007,33.0,astrocytoma,male,NaN,white,nothispanicorlatino,1915.0,1.0,"1915,1",TCGA.DU.7007
351,476,TCGA.TQ.A7RM,41.0,oligoastrocytoma,female,yes,white,hispanicorlatino,1116.0,0.0,"1116,0",TCGA.TQ.A7RM
352,322,TCGA.HT.A61C,66.0,oligodendroglioma,male,yes,white,nothispanicorlatino,537.0,1.0,"537,1",TCGA.HT.A61C
353,382,TCGA.QH.A6CZ,38.0,oligoastrocytoma,male,no,white,nothispanicorlatino,279.0,0.0,"279,0",TCGA.QH.A6CZ


In [36]:
train_clinical73=train_clinical73.drop("Unnamed: 0_x",axis=1)

In [39]:
test_clinical73=test_clinical.drop("Unnamed: 0",axis=1)

In [ ]:
test_clinical73.to_csv("/Users/vivit/CC/test_clinical_B73.csv")

In [40]:
trainset1=trainset.drop("Unnamed: 0",axis=1)
testset1=testset.drop("Unnamed: 0",axis=1)

In [41]:
trainset2=trainset1.drop("Unnamed: 0.1",axis=1)
testset2=testset1.drop("Unnamed: 0.1",axis=1)

In [42]:
trainset2

,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,AAA1,AAAS,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
0,6.1944,0.0000,9.0375,7.1808,9.0290,14.1314,7.3288,1.0252,0.0000,8.4955,...,6.7406,8.8721,10.2618,0.4277,10.7750,11.9259,10.7387,9.9894,4.3204,0.0
1,6.2375,0.0000,8.5295,5.7117,7.4545,14.2341,5.5011,0.4287,0.0000,9.1342,...,6.8608,8.8308,10.1078,0.4287,10.6322,11.1134,10.4766,9.4674,10.7374,0.0
2,4.9535,0.0000,5.5235,3.9052,7.4920,13.5168,5.1915,1.2043,0.0000,10.2793,...,6.4921,8.9427,10.3324,2.0911,10.7823,11.0584,10.9054,9.4311,8.0716,0.0
3,4.6844,0.0000,6.7335,4.2491,7.3253,13.4115,6.8579,0.5892,0.0000,9.0637,...,5.4385,8.3434,9.9855,0.5892,10.2598,10.7259,9.5133,8.8390,4.3694,0.0
4,5.5662,0.0000,8.3424,4.9114,7.3717,14.0085,6.2716,0.0000,0.0000,9.5095,...,5.7292,8.6205,10.3663,0.0000,10.6882,10.7732,10.9516,9.7250,9.7346,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,7.7583,0.0000,2.6741,5.3648,5.5743,15.3476,7.9331,0.0000,0.0000,9.9082,...,5.2734,8.2489,10.0079,0.6763,9.8439,11.4981,10.3045,8.8761,9.2236,0.0
351,6.3097,0.0000,7.4914,4.8743,8.3211,13.1848,5.7910,0.6115,0.0000,9.8725,...,5.7772,8.4595,10.3039,0.0000,10.5437,10.5493,10.6046,9.3784,3.0686,0.0
352,6.4383,0.0000,5.4604,4.9876,5.3276,14.1384,5.5559,1.4442,0.0000,9.4867,...,5.9364,8.9773,10.3973,0.8958,10.4965,10.6219,10.7201,9.7107,12.8674,0.0
353,6.9881,0.4944,5.5221,4.5847,5.1757,13.4168,6.5510,1.1545,0.0000,9.8779,...,6.2746,9.0549,10.6047,1.9488,10.8586,10.9134,10.9709,9.8315,11.5579,0.0


In [43]:
trainset3=trainset2.to_numpy()
testset3=testset2.to_numpy()

In [44]:
# standardize inputs
MMScaler=MinMaxScaler()
train_scaler=MMScaler.fit_transform(trainset3)
test_scaler=MMScaler.fit_transform(testset3)

In [45]:
train_torch = torch.FloatTensor(train_scaler)
train_loader = torch.utils.data.DataLoader(train_torch, batch_size = 256, shuffle=True)
test_torch = torch.FloatTensor(test_scaler)
test_loader = torch.utils.data.DataLoader(test_torch, batch_size = 256, shuffle=False)

In [46]:
# define parameters
EPOCH=10
lr=0.0001

In [47]:
train_torch.shape

torch.Size([355, 20118])

In [48]:
test_torch.shape

torch.Size([153, 20118])

In [50]:
class Autoencoder512(nn.Module):
    def __init__(self):
        super(Autoencoder512, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=20118, out_features=2048),
            nn.ReLU(),
            nn.Linear(in_features=2048, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=512)  
            
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=512, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=2048),
            nn.ReLU(),
            nn.Linear(in_features=2048, out_features=20118)
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return decoded
    
    
model512 = Autoencoder512()
if torch.cuda.is_available():
    model512.cuda()
    
model512.train(mode=True)
# define optimizer
optimizer512 = torch.optim.Adam(model512.parameters(), lr=lr)
criterion512=nn.MSELoss()


In [53]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch:
        output = model512(batch_features)
        loss = criterion512(output, batch_features)
        optimizer512.zero_grad()
        loss.backward()
        optimizer512.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0211
epoch [2/10], loss:0.0194
epoch [3/10], loss:0.0200
epoch [4/10], loss:0.0191
epoch [5/10], loss:0.0192
epoch [6/10], loss:0.0192
epoch [7/10], loss:0.0193
epoch [8/10], loss:0.0184
epoch [9/10], loss:0.0186
epoch [10/10], loss:0.0171


In [51]:
feature=model512.encoder(train_torch)
feature

tensor([[ 0.0703, -0.0912,  0.0236,  ..., -0.0621, -0.0466, -0.0146],
        [ 0.0801, -0.0862,  0.0135,  ..., -0.0530, -0.0388,  0.0027],
        [ 0.0835, -0.0799,  0.0278,  ..., -0.0454, -0.0617, -0.0005],
        ...,
        [ 0.0986, -0.0965,  0.0207,  ..., -0.0262, -0.0482,  0.0035],
        [ 0.1003, -0.0985,  0.0169,  ..., -0.0343, -0.0339, -0.0065],
        [ 0.0771, -0.1024,  0.0116,  ..., -0.0325, -0.0392,  0.0004]],
       grad_fn=<AddmmBackward>)

In [53]:
a = feature.detach().numpy()
print(a)

[[ 0.07027124 -0.09120511  0.02359719 ... -0.06210298 -0.04662137
  -0.01456599]
 [ 0.08010261 -0.08618077  0.01351747 ... -0.05304248 -0.03882463
   0.00272085]
 [ 0.08351094 -0.07993254  0.02780074 ... -0.04544431 -0.06166463
  -0.00045349]
 ...
 [ 0.09855156 -0.09648471  0.02071491 ... -0.02619267 -0.04820897
   0.00347204]
 [ 0.10033759 -0.09850293  0.01686999 ... -0.03427501 -0.03388095
  -0.00646848]
 [ 0.07713401 -0.10243759  0.01159581 ... -0.03247996 -0.03916603
   0.00042939]]


In [54]:
a.shape

(355, 512)

In [19]:
np.savetxt("/Users/vivit/Capstone/RNAseq AE results for Glioma Cancer 73.csv",a,delimiter=',')

In [70]:
torch.save(model128, '/Users/vivit/Capstone/Glioma/AE model for RNAseq 73.pkl')

In [54]:
class Autoencoder256(nn.Module):
    def __init__(self):
        super(Autoencoder256, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=20118, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=256),
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=256, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=20118),
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return encoded,decoded
    
    
model256 = Autoencoder256()
if torch.cuda.is_available():
    model256.cuda()
    
model256.train(mode=True)
# define optimizer
optimizer256 = torch.optim.Adam(model256.parameters(), lr=lr)
criterion256=nn.MSELoss()

In [55]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch:
        _,output = model256(batch_features)
        loss = criterion256(output, batch_features)    
        optimizer256.zero_grad()
        loss.backward()
        optimizer256.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0208
epoch [2/10], loss:0.0189
epoch [3/10], loss:0.0188
epoch [4/10], loss:0.0189
epoch [5/10], loss:0.0182
epoch [6/10], loss:0.0168
epoch [7/10], loss:0.0157
epoch [8/10], loss:0.0155
epoch [9/10], loss:0.0154
epoch [10/10], loss:0.0158


In [56]:
class Autoencoder128(nn.Module):
    def __init__(self):
        super(Autoencoder128, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=20118, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128)
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=128, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=20118)
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return encoded,decoded
    
    
model128 = Autoencoder128()
if torch.cuda.is_available():
    model128.cuda()
    
model128.train(mode=True)
# define optimizer
optimizer128 = torch.optim.Adam(model128.parameters(), lr=lr)
criterion128=nn.MSELoss()

In [57]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch:
        _,output = model128(batch_features)
        loss = criterion128(output, batch_features)    
        optimizer128.zero_grad()
        loss.backward()
        optimizer128.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0202
epoch [2/10], loss:0.0189
epoch [3/10], loss:0.0178
epoch [4/10], loss:0.0183
epoch [5/10], loss:0.0181
epoch [6/10], loss:0.0179
epoch [7/10], loss:0.0167
epoch [8/10], loss:0.0156
epoch [9/10], loss:0.0150
epoch [10/10], loss:0.0145


In [58]:
class Autoencoder64(nn.Module):
    def __init__(self):
        super(Autoencoder64, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=20118, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=64, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=20118),
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return encoded,decoded
    
    
model64 = Autoencoder64()
if torch.cuda.is_available():
    model64.cuda()
    
model64.train(mode=True)
# define optimizer
optimizer64 = torch.optim.Adam(model64.parameters(), lr=lr)
criterion64=nn.MSELoss()

In [59]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch:
        _,output = model64(batch_features)
        loss = criterion64(output, batch_features)    
        optimizer64.zero_grad()
        loss.backward()
        optimizer64.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0193
epoch [2/10], loss:0.0190
epoch [3/10], loss:0.0185
epoch [4/10], loss:0.0176
epoch [5/10], loss:0.0175
epoch [6/10], loss:0.0175
epoch [7/10], loss:0.0176
epoch [8/10], loss:0.0176
epoch [9/10], loss:0.0176
epoch [10/10], loss:0.0172


In [28]:
from sklearn.model_selection import KFold,StratifiedKFold
KF=KFold(n_splits=10,shuffle=True,random_state=123)
print(KF)

KFold(n_splits=10, random_state=123, shuffle=True)


In [62]:
def train_epoch(model,device,dataloader,loss_fun,optimizer):
    train_loss=0.0
    model.train()
    for x in dataloader:
        x=x.to(device)
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fun(output,x)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss=train_loss/len(dataloader)
    return train_loss
def valid_epoch(model,device,dataloader,loss_fun):
    valid_loss = 0.0
    model.eval()
    for x in dataloader:
        x=x.to(device)
        output = model(x)
        loss=loss_func(output,x)
        valid_loss+=loss.item()
    valid_loss+=valid_loss/len(dataloader)
    return valid_loss

In [65]:
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset 
K_valid_loss=[]
for fold, (train_idx,val_idx) in enumerate(KF.split(np.arange(len(test_torch)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = torch.utils.data.DataLoader(train_torch,batch_size=512,sampler=train_sampler)
    test_loader = torch.utils.data.DataLoader(test_torch,batch_size=512,sampler=test_sampler)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Autoencoder512()
    model.to(device)
    loss_func=nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    history = {'train_loss': [], 'test_loss': []}
    for epoch in range(EPOCH):
        train_loss=train_epoch(model,device,train_loader,criterion,optimizer)
        test_loss=valid_epoch(model,device,test_loader,criterion)
        print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} ".format(epoch + 1,EPOCH,train_loss, test_loss))

Fold 1
Epoch:1/20 AVG Training Loss:0.246 AVG Test Loss:0.664 
Epoch:2/20 AVG Training Loss:0.328 AVG Test Loss:0.428 
Epoch:3/20 AVG Training Loss:0.211 AVG Test Loss:0.226 
Epoch:4/20 AVG Training Loss:0.108 AVG Test Loss:2.162 
Epoch:5/20 AVG Training Loss:1.046 AVG Test Loss:0.151 
Epoch:6/20 AVG Training Loss:0.069 AVG Test Loss:0.331 
Epoch:7/20 AVG Training Loss:0.162 AVG Test Loss:0.400 
Epoch:8/20 AVG Training Loss:0.196 AVG Test Loss:0.411 
Epoch:9/20 AVG Training Loss:0.202 AVG Test Loss:0.395 
Epoch:10/20 AVG Training Loss:0.194 AVG Test Loss:0.354 
Epoch:11/20 AVG Training Loss:0.174 AVG Test Loss:0.279 
Epoch:12/20 AVG Training Loss:0.136 AVG Test Loss:0.178 
Epoch:13/20 AVG Training Loss:0.085 AVG Test Loss:0.176 
Epoch:14/20 AVG Training Loss:0.083 AVG Test Loss:0.150 
Epoch:15/20 AVG Training Loss:0.069 AVG Test Loss:0.082 
Epoch:16/20 AVG Training Loss:0.035 AVG Test Loss:0.092 
Epoch:17/20 AVG Training Loss:0.040 AVG Test Loss:0.096 
Epoch:18/20 AVG Training Loss:0.0